In [1]:
import pandas as pd
import pprint
from pathlib import Path
from geopy.geocoders import Nominatim
import folium

In [2]:
df = pd.read_csv("Cleaned_Bridge_df_Ver1.csv")
df

,Latitude,Longitude,Year Built,Maintenance Responsibility,Bridge Condition,SqFt,Repair Cost Est,Replacement Cost Est
0,40.103450,-74.961092,1990,City or Municipal Highway Agency,G,237.90,74700.60,9.860479e+04
1,40.017092,-75.211428,1928,City or Municipal Highway Agency,F,631.68,198347.52,2.618187e+05
2,39.963467,-75.186778,1964,City or Municipal Highway Agency,P,2688.70,844251.80,1.114412e+06
3,39.963861,-75.185025,1964,City or Municipal Highway Agency,P,439.02,137852.28,1.819650e+05
4,40.035356,-75.208878,1957,City or Municipal Highway Agency,F,2384.64,748776.96,9.883856e+05
...,...,...,...,...,...,...,...,...
153,39.958900,-75.170372,2017,City or Municipal Highway Agency,G,592.02,185894.28,2.453804e+05
154,39.959400,-75.173906,2018,City or Municipal Highway Agency,G,795.60,249818.40,3.297603e+05
155,39.959500,-75.175500,2018,City or Municipal Highway Agency,G,838.95,263430.30,3.477280e+05
156,39.960194,-75.159694,1892,City or Municipal Highway Agency,P,82.08,25773.12,3.402052e+04


In [3]:
geolocator = Nominatim(user_agent="bridge_address")

In [4]:
def reverse_geocode(row):
    location = geolocator.reverse((row["Latitude"], row['Longitude']), exactly_one=True)
    if location:
        return location.address
    else:
        return "N/A"
    
#Add in Address column with reverse geocode data
df["Address"] = df.apply(reverse_geocode, axis=1)

df

,Latitude,Longitude,Year Built,Maintenance Responsibility,Bridge Condition,SqFt,Repair Cost Est,Replacement Cost Est,Address
0,40.103450,-74.961092,1990,City or Municipal Highway Agency,G,237.90,74700.60,9.860479e+04,"Dunksferry Road, Mechanicsville, Bensalem Town..."
1,40.017092,-75.211428,1928,City or Municipal Highway Agency,F,631.68,198347.52,2.618187e+05,"Ridge Avenue, Wissahickon, Philadelphia, Phila..."
2,39.963467,-75.186778,1964,City or Municipal Highway Agency,P,2688.70,844251.80,1.114412e+06,"Spring Garden Street, Mantua, Philadelphia, Ph..."
3,39.963861,-75.185025,1964,City or Municipal Highway Agency,P,439.02,137852.28,1.819650e+05,"2917, Spring Garden Street, Mantua, Philadelph..."
4,40.035356,-75.208878,1957,City or Municipal Highway Agency,F,2384.64,748776.96,9.883856e+05,"Henry Avenue, Philadelphia, Philadelphia Count..."
...,...,...,...,...,...,...,...,...,...
153,39.958900,-75.170372,2017,City or Municipal Highway Agency,G,592.02,185894.28,2.453804e+05,"North 19th Street, Center City, Philadelphia, ..."
154,39.959400,-75.173906,2018,City or Municipal Highway Agency,G,795.60,249818.40,3.297603e+05,"Vine Street Expressway, Center City, Philadelp..."
155,39.959500,-75.175500,2018,City or Municipal Highway Agency,G,838.95,263430.30,3.477280e+05,"North 22nd Street, Center City, Philadelphia, ..."
156,39.960194,-75.159694,1892,City or Municipal Highway Agency,P,82.08,25773.12,3.402052e+04,"Noble Street, Chinatown, Center City, Philadel..."


In [11]:
myMap = folium.Map(location = (39.952583, -75.165222), tiles = "OpenStreetMap")
myMap

In [12]:
folium.TileLayer("cartodbpositron").add_to(myMap)

In [13]:
condition = df["Bridge Condition"]
def get_marker_color(condition):
    if condition == "G":
        return "green"
    elif condition == "F":
        return "orange"
    elif condition == "P":
        return "darkred"
    else:
        return "gray"

#Create separate groups for each marker type
good_group = folium.FeatureGroup(name="Good (G)")
fair_group = folium.FeatureGroup(name="Fair (F)")
poor_group = folium.FeatureGroup(name="Poor (P)")

#Markers
for index, row in df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    condition = row["Bridge Condition"]
    address = row["Address"]
    #Marker Coloring
    marker_color = get_marker_color(condition)
    text = f"Address: {address}<br>Latitude: {lat} <br>Longitude: {long} <br>Condition: {condition}"
    popup = folium.Popup(text, max_width = 300)
    marker = folium.Marker([lat, long], popup=popup, icon=folium.Icon(color=marker_color))
    
    if condition == "G":
        marker.add_to(good_group)
    elif condition == "F":
        marker.add_to(fair_group)
    elif condition == "P":
        marker.add_to(poor_group)
    else:
        marker.add_to(myMap)

good_group.add_to(myMap)
fair_group.add_to(myMap)
poor_group.add_to(myMap)

folium.LayerControl().add_to(myMap)

myMap

In [8]:
myMap.save('map.html')